# Project Pipeline Lab


## 1. Data loading
In this lab we will create pipelines for data processing on the [Titanic dataset](http://www.kaggle.com/c/titanic-gettingStarted/data).

The dataset is a list of passengers. The second column of the dataset is a “label” for each person indicating whether that person survived (1) or did not survive (0). Here is the Kaggle page with more information on the dataset:

You can grab the titanic data as follows:

    psql -h dsi.c20gkj5cvu3l.us-east-1.rds.amazonaws.com -p 5432 -U dsi_student titanic
    password: gastudents

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sqlalchemy import create_engine
import pandas as pd
from pandas.io import sql
import sqlite3
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn.preprocessing import scale
from sklearn.feature_extraction import DictVectorizer

In [2]:
conn = sqlite3.connect('../../assets/datasets/train.csv')
c = conn.cursor()
conn.commit()




In [3]:
df = pd.read_csv('../../assets/datasets/train.csv')
df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


Have a look at the data using the info method:

- Are there numerical features?
- Are there categorical features?
- Which columns have missing data?
- Which of these are important to be filled?

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [5]:
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [6]:
df.Sex.value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [7]:
#df.Cabin.value_counts()

In [8]:
df["Embarked"].value_counts()
df["Embarked"].fillna("S")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [9]:
#df.isnull() # age ,cabin, embarked


In [10]:
#df.Age

## 2. Age

Several passengers are missing data points for age. Impute the missing values so that there are no “NaN” values for age as inputs to your model. Explore the distribution of age and decide how you are going to impute the data.

In [11]:
#df["preTestScore"].fillna(df["preTestScore"].mean(), inplace=True)
df["Age"].fillna(df["Age"].mean(),inplace = True)


### 2.b Age Transformer

Create a custom transformer that imputes the age values. Depending on how you have decided to impute missing values, this could involve:

- Selecting one or more columns
- Filling the NAs using Imputer or a custom strategy
- Scaling the Age values

In [35]:
age_pipe = df["Age"]

In [38]:
age_pipe.fit_transform(df.head())

AttributeError: 'Series' object has no attribute 'fit_transform'

## 3. Categorical Variables

`Embarked` and `Pclass` are categorical variables. Use pandas get_dummies function to create dummy columns corresponding to the values.

`Embarked` has 2 missing values. Fill them with the most common port of embarkment.

Feel free to create a GetDummiesTransformer that wraps around the get_dummies function.

In [13]:
em_dm = pd.get_dummies(df['Embarked'])
em_dm[0:3]


,C,Q,S
0,0.0,0.0,1.0
1,1.0,0.0,0.0
2,0.0,0.0,1.0


In [14]:
df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [15]:
p_dm = pd.get_dummies(df['Pclass'])
p_dm[0:3]

,1,2,3
0,0.0,0.0,1.0
1,1.0,0.0,0.0
2,0.0,0.0,1.0


In [16]:
df['Pclass'].value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [17]:
# one_hot_pipe.fit_transform(df.head())

## 4. Boolean Columns

The `Sex` column only contains 2 values: `male` and `female`. Build a custom transformers that is initialized with one of the values and returns a boolean column with values of `True` when that value is found and `False` otherwise.

In [22]:
df.loc[:,"Sex"] = df.loc[:,"Sex"].astype(str)
df['Sex'].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [26]:
df["class"] = df['Sex'].str.contains("female")
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,class
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,NaN,S,False
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C85,C,True
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,S,True
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,C123,S,True
4,5,0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,NaN,S,False
5,6,0,3,"Moran, Mr. James",male,29.699118,0,0,330877,8.4583,NaN,Q,False
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.000000,0,0,17463,51.8625,E46,S,False
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.000000,3,1,349909,21.0750,NaN,S,False
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.000000,0,2,347742,11.1333,NaN,S,True
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.000000,1,0,237736,30.0708,NaN,C,True


In [27]:
df["class"].value_counts()

False    577
True     314
Name: class, dtype: int64

In [21]:
# gender_pipe.fit_transform(df.head())

## 5. Fare

The `Fare` attribute can be scaled using one of the scalers from the preprocessing module. 

In [33]:
# fare_pipe.fit_transform(df.head())
#preprocessing.scale(X)
scale(df["Fare"])
df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,class
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,False
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,True


## 6. Union

Use the `make_union` function from the `sklearn.pipeline` modeule to combine all the pipes you have created.

In [ ]:
# union.fit_transform(df.head())

The union you have created is a complete pre-processing pipeline that takes the original titanic dataset and extracts a bunch of features out of it. The last step of this process is to persist the `union` object to disk, so that it can be used again later. The following lines achieve this:

In [ ]:
import dill
import gzip

with gzip.open('../../assets/datasets/union.dill.gz', 'w') as fout:
    dill.dump(union, fout)

## Bonus

Can you think of a way to engineer an additional boolean feature that keeps track whethere the person is travelling alone or with family?